<a href="https://colab.research.google.com/github/hussain0048/Natural-language-processing/blob/main/Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

This Portion is intended to test all the dependencies that are needed for the rest of the notebooks to run properly. If everything is tested correctly you should see a "You are all set" message when running the code below

In [ ]:
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
import string
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from pprint import pprint
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from nltk.corpus import reuters
from operator import itemgetter

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Test that verifies that the imports don't fail and that we have the correct files from nltk downloaded
reuters.fileids()
stopwords.words('english')
word_tokenize('This is just a test')

print('You are all set')

You are all set


# **1- Text Classification**

## **1.1-Binary text classification**

We will address the binary problem of detecting Sport related documents vs any other type of documents. In order to do this we will create an artificial (and very small collection).

Define a set of labelled documents that will be our training dataset. These are the documents the classifier will learn from in order to categorise future unseen documents

Define a set of labelled documents that will be our testing dataset. These will be the "unseen" documents that the classifier will predict (without having being trained with them)

Represent our training and testing documents

Train the classifier based on the training data

Predict the labels for the testing documents

###**1.1.1-Support Vector Machine (SVM)**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [ ]:
# Train and test data. Both the full documents and their labels ("Sports" vs "Non Sports")
train_data = ['Football: a great sport', 'The referee has been very bad this season', 'Our team scored 5 goals', 'I love tennis',
              'Politics is in decline in the UK', 'Brexit means Brexit', 'The parlament wants to create new legislation',
              'I so want to travel the world']
train_labels = ["Sports","Sports","Sports","Sports", "Non Sports", "Non Sports", "Non Sports", "Non Sports"]

test_data = ['Swimming is a great sport',
             'A lot of policy changes will happen after Brexit',
             'The table tennis team will travel to the UK soon for the European Championship']
test_labels = ["Sports","Non Sports","Sports"]

In [ ]:
# Representation of the data using TF-IDF
vectorizer = TfidfVectorizer()
vectorised_train_data = vectorizer.fit_transform(train_data)
vectorised_test_data = vectorizer.transform(test_data)

# Train the classifier given the training data
classifier = LinearSVC()
classifier.fit(vectorised_train_data, train_labels)

# Predict the labels for the test documents (not used for training)
print(classifier.predict(vectorised_test_data))

['Sports' 'Non Sports' 'Non Sports']


However, the third case is wrongly classified. Why do you think that might be?

Matching problems (e.g., "car" is different than "Cars")

Cases never seen before (e.g., the classifier has never seen the word "table")

"Spurious" correlations and bias ("car" appears only in the positive category)

Lets look into how we are representing our documents



In [ ]:
from pprint import pprint

# Function to show the feature weights of a document (to be explained later)
def feature_values(doc, representer):
    doc_representation = representer.transform([doc])
    features = representer.get_feature_names()
    return [(features[index], doc_representation[0, index]) for index in doc_representation.nonzero()[1]]

pprint([feature_values(doc, vectorizer) for doc in test_data])

**Lets try again, with stop-word removal this time**

In [ ]:
from nltk.corpus import stopwords

# Load the list of (english) stop-words from nltk
stop_words = stopwords.words("english")

# Represent, train, predict
vectorizer = TfidfVectorizer(stop_words=stop_words)
vectorised_train_data = vectorizer.fit_transform(train_data)
vectorised_test_data = vectorizer.transform(test_data)
classifier = LinearSVC()
classifier.fit(vectorised_train_data, train_labels)

print(classifier.predict(vectorised_test_data))
# Expected: [Sports, Non Sports, Sports]

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: ignored

## **1.2-Multi-Class classification problem**

### **1.2.1-Support Vector Machine (SVM)**

We will address the multi-class problem of detecting the language of a sentence based on 3 mutually exclusive languages (e.g., Spanish, English and French). For the sake of this example, we assume those are the only 3 languages that the documents can have. As before, we will create an artificial (and very small collection) with similar steps

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [ ]:
# Artificial (and small) dataset. Spanish,English,French texts
train_data = ['PyCon es una gran conferencia', 'Aprendizaje automatico esta listo para dominar el mundo dentro de poco',
             'This is a great conference with a lot of amazing talks', 'AI will dominate the world in the near future',
             'Dix chiffres por resumer le feuilleton de la loi travail']
train_labels = ["SP", "SP", "EN", "EN", "FR"]

test_data = ['Estoy preparandome para dominar las olimpiadas', 'Me gustaria mucho aprender el lenguage de programacion Scala',
             'Machine Learning is amazing','Hola a todos']
test_labels = ["SP", "SP", "EN", "SP"]

In [ ]:
# Represent
vectorizer = TfidfVectorizer() # Note, we are not doing stop-word removal. Stop words could be beneficial in this problems
vectorised_train_data = vectorizer.fit_transform(train_data)
vectorised_test_data = vectorizer.transform(test_data)


In [ ]:
# Train
classifier = LinearSVC()
classifier.fit(vectorised_train_data, train_labels)

In [ ]:
from pprint import pprint

In [ ]:
# Predict
predictions = classifier.predict(vectorised_test_data)
pprint(predictions)

array(['SP', 'SP', 'EN', 'EN'], dtype='<U2')


The last case is wrong. Can you guess why?

Can we learn from never seen cases?


We will address the multi-label problem of labeling documents as being relevant to Sports or Politics. As before, we will create an artificial (and very small collection) with initial similar steps.

There are two modifications for our example to run in a multi-label way:

- Change the representation of the data viewing every document as a list of bits, representing being or not to each category. (MultiLabelBinarizer)

- Run the classifier N times, once for each category where the negative cases will be the documents in all the other categories. (OneVsRestClassifier)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
# Artificial (and small) dataset. Sports and Politics
train_data = ['Football: a great sport', 'The referee has been very bad this season', 'Our team scored 5 goals', 'I love tennis',
              'Politics is in decline in the UK', 'Brexit means Brexit', 'The parlament wants to create new legislation',
              'I so want to travel the world',
              'The goverment will increase the budget for sports in the UK after the victories in the Olimpic Games',
              "O'Reilly has a great conference this year"]
train_labels = [["Sports"], ["Sports"], ["Sports"], ["Sports"],["Politics"],["Politics"],["Politics"],[],["Politics", "Sports"],[]]

test_data = ['Swimming is a great sport',
             'A lot of policy changes will happen after Brexit',
             'The table tennis team will travel to the UK soon for the European Championship',
             'The goverment will increase the budget for sports in the UK after the victories in the Olimpic Games',
             'PyCon is my favourite conference']
test_labels = [["Sports"], ["Politics"], ["Sports"], ["Politics","Sports"],[]]

In [ ]:
# Change the representation of our data as a list of bit lists
mlb = MultiLabelBinarizer()
binary_train_labels = mlb.fit_transform(train_labels)
binary_test_labels = mlb.transform(test_labels)

print(binary_train_labels)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords

# Load the list of (english) stop-words from nltk
stop_words = stopwords.words("english")

In [ ]:
# Represent
vectorizer = TfidfVectorizer(stop_words=stop_words)
vectorised_train_data = vectorizer.fit_transform(train_data)
vectorised_test_data = vectorizer.transform(test_data)


In [ ]:
# One classifer built per category using a one vs the rest approach
classifier = OneVsRestClassifier(LinearSVC())
classifier.fit(vectorised_train_data, binary_train_labels)

OneVsRestClassifier(estimator=LinearSVC())

In [ ]:
#Predict
predictions = classifier.predict(vectorised_test_data)

In [ ]:
print(predictions)
print()

[[0 1]
 [1 0]
 [0 1]
 [1 1]
 [0 0]]



In [ ]:
print(mlb.inverse_transform(predictions))


### **1.2.2-Naive Bayes classifie**

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [3]:
# Load the 20 Newsgroups dataset
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [ ]:
newsgroups_test

In [4]:
# Transform the texts into TF-IDF vectors
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_test = vectorizer.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [ ]:
# Train a Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

In [8]:
# Predict the newsgroup of the test texts
y_pred = clf.predict(X_test)

In [9]:
# Evaluate the classifier's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 77.39%


This code will load the 20 Newsgroups dataset and split it into training and test sets. Then it will transform the texts into numerical representation using TfidfVectorizer and train a Multinomial Naive Bayes classifier using the training set. Finally, it will use the trained classifier to predict the newsgroup of the test texts and evaluate the classifier’s accuracy.

## **1.2-Sentiment Analysis**


In [12]:
from nltk.corpus import stopwords

# Load the list of (english) stop-words from nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [15]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Define text to be analyzed
text = "I love this product, it's amazing"

# Get sentiment score
sentiment_score = sia.polarity_scores(text)

# Print sentiment score
print(sentiment_score)

# Classify sentiment as positive, negative, or neutral
if sentiment_score['compound'] > 0.5:
    print("Positive sentiment")
elif sentiment_score['compound'] < -0.5:
    print("Negative sentiment")
else:
    print("Neutral sentiment")

{'neg': 0.0, 'neu': 0.273, 'pos': 0.727, 'compound': 0.8402}
Positive sentiment


# **2-Machine Translation**

In [ ]:
!pip install  http://download.pytorch.org/whl/cu80/torch-0.1.9.post2-cp27-none-linux_x86_64.whl


In [16]:
from opennmt import tokenizers
from opennmt import models
import torch

# Tokenize the source and target text.
source_tokenizer = tokenizers.new("text", "en")
source_text = "Hello, how are you?"
source_tokens = source_tokenizer.tokenize(source_text)

target_tokenizer = tokenizers.new("text", "fr")
target_text = "Bonjour, comment vas-tu?"
target_tokens = target_tokenizer.tokenize(target_text)

# Build the translation model.
model = models.Transformer(
    source_vocab_size=len(source_tokenizer.vocab),
    target_vocab_size=len(target_tokenizer.vocab),
    num_layers=6,
    hidden_size=512,
    dropout=0.1,
    attention_dropout=0.1,
    relu_dropout=0.1)
model.eval()

# Convert the tokens to a tensor.
source_tokens = torch.tensor(source_tokenizer.encode(source_text)).unsqueeze(0)

# Generate a translation.
with torch.no_grad():
    log_probs, _, _ = model(source_tokens, None, None)
    tokens = log_probs.argmax(-1)

# Decode the translation.
translation = target_tokenizer.decode(tokens[0])
print(translation)

ModuleNotFoundError: ignored

# **References**

[1-nlp-tutorial](https://github.com/bonzanini/nlp-tutorial/blob/master/notebooks/00%20Environment%20Test.ipynb)